# 02_Data_Preprocessing: Cleaning and Combining Global Health Data

## Objective

The goal of this notebook is to construct a unified and analysis-ready dataset for studying global mortality trends. We begin with the IHME Global Burden of Disease dataset as our core and incrementally enrich it with auxiliary data from healthcare infrastructure, economic investment, and governance sources.

---

## Workflow Plan

1. **Load and clean the IHME mortality dataset**  
   - Keep only mortality *rate*  
   - Remove irrelevant columns  
   - Standardize country and year columns  
   - This becomes our `final_df`

2. **Clean each auxiliary dataset one by one**  
   - Health spending per capita  
   - Percentage of GDP on health  
   - Hospital beds per 1,000 people  
   - Physicians per 1,000 people  
   - Corruption index  
   - Life satisfaction score

3. **Merge each cleaned dataset into `final_df`**  
   - Using `country` and `year` as join keys  
   - Only merge when columns are standardized

4. **Final cleanup**  
   - Handle missing values  
   - Save as `final_combined_dataset.csv`

---

## Notes

- No filtering or modeling is performed here  
- All transformations are done in memory, with no intermediate file writes  
- Column name consistency and alignment across countries is a key focus


In [32]:
import pandas as pd

# Load the IHME dataset
file_path = "../Data/IHME-GBD_2021_DATA-cc52e0a9-1.csv"  # Update path if needed
df_ihme = pd.read_csv(file_path)

# Select only the necessary columns
columns_required = [
    'location_id', 'location_name', 'cause_id',
    'cause_name', 'metric_name', 'year', 'val'
]
df_ihme = df_ihme[columns_required]

# Filter rows to include only metric_name == "Rate"
df_ihme = df_ihme[df_ihme['metric_name'].str.lower() == 'rate']

# View basic info
print(df_ihme.info())
print(df_ihme.head())


<class 'pandas.core.frame.DataFrame'>
Index: 51408 entries, 2 to 154223
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   location_id    51408 non-null  int64  
 1   location_name  51408 non-null  object 
 2   cause_id       51408 non-null  int64  
 3   cause_name     51408 non-null  object 
 4   metric_name    51408 non-null  object 
 5   year           51408 non-null  int64  
 6   val            51408 non-null  float64
dtypes: float64(1), int64(3), object(3)
memory usage: 3.1+ MB
None
    location_id location_name  cause_id                            cause_name  \
2           150          Oman       717  Self-harm and interpersonal violence   
5            36    Kazakhstan       526                    Digestive diseases   
8            36    Kazakhstan       542                Neurological disorders   
11           20      Viet Nam       410                             Neoplasms   
14           19   Timor-Lest

In [33]:
# Rename columns
df_ihme = df_ihme.rename(columns={
    'val': 'mortality_rate',
    'location_id': 'country_id',
    'location_name': 'country_name'
})

# Drop the 'metric_name' column (if not already dropped)
df_ihme = df_ihme.drop(columns=['metric_name'], errors='ignore')

# Preview the cleaned DataFrame
print(df_ihme.head())


    country_id country_name  cause_id                            cause_name  \
2          150         Oman       717  Self-harm and interpersonal violence   
5           36   Kazakhstan       526                    Digestive diseases   
8           36   Kazakhstan       542                Neurological disorders   
11          20     Viet Nam       410                             Neoplasms   
14          19  Timor-Leste       626             Musculoskeletal disorders   

    year  mortality_rate  
2   2010        2.046552  
5   2010       54.897349  
8   2010       18.051611  
11  2010       77.569869  
14  2010        0.588912  


In [34]:
import pycountry

# Function to get 3-letter ISO code (capitalized)
def get_iso3_code(name):
    try:
        return pycountry.countries.lookup(name).alpha_3.upper()
    except LookupError:
        return None

# Manual overrides for unmatched countries
manual_corrections = {
    "Bolivia (Plurinational State of)": "BOL",
    "Democratic Republic of the Congo": "COD",
    "Iran (Islamic Republic of)": "IRN",
    "Micronesia (Federated States of)": "FSM",
    "Palestine": "PSE",
    "Republic of Korea": "KOR",
    "Türkiye": "TUR",
    "United States Virgin Islands": "VIR",
    "Venezuela (Bolivarian Republic of)": "VEN"
}

# Apply ISO codes using both methods
df_ihme['country_code'] = df_ihme['country_name'].apply(get_iso3_code)
df_ihme['country_code'] = df_ihme.apply(
    lambda row: manual_corrections.get(row['country_name'], row['country_code']),
    axis=1
)

# Sort by country name
df_ihme = df_ihme.sort_values(by='country_name').reset_index(drop=True)

# Preview distinct country-code pairs
print(df_ihme[['country_name', 'country_code']].drop_duplicates().sort_values('country_name').to_string(index=False))


                         country_name country_code
                          Afghanistan          AFG
                              Albania          ALB
                              Algeria          DZA
                       American Samoa          ASM
                              Andorra          AND
                               Angola          AGO
                  Antigua and Barbuda          ATG
                            Argentina          ARG
                              Armenia          ARM
                            Australia          AUS
                              Austria          AUT
                           Azerbaijan          AZE
                              Bahamas          BHS
                              Bahrain          BHR
                           Bangladesh          BGD
                             Barbados          BRB
                              Belarus          BLR
                              Belgium          BEL
                               

In [35]:
print(df_ihme.head())

   country_id country_name  cause_id                       cause_name  year  \
0         160  Afghanistan       688               Transport injuries  2010   
1         160  Afghanistan       973          Substance use disorders  2014   
2         160  Afghanistan       962  Maternal and neonatal disorders  2014   
3         160  Afghanistan       961        Other infectious diseases  2014   
4         160  Afghanistan       653   Skin and subcutaneous diseases  2014   

   mortality_rate country_code  
0       27.601971          AFG  
1        1.407217          AFG  
2       78.941246          AFG  
3       48.133560          AFG  
4        0.511941          AFG  


In [36]:
# Reorder columns in the required sequence
df_ihme = df_ihme[[
    'country_id', 'country_name', 'country_code',
    'cause_id', 'cause_name', 'year', 'mortality_rate'
]]

# This is now our master dataset
df_final = df_ihme.copy()

# Save to CSV as a clean reference copy (not for intermediate use)
df_final.to_csv("../Data/final.csv", index=False)

# Preview
print(df_final.head())


   country_id country_name country_code  cause_id  \
0         160  Afghanistan          AFG       688   
1         160  Afghanistan          AFG       973   
2         160  Afghanistan          AFG       962   
3         160  Afghanistan          AFG       961   
4         160  Afghanistan          AFG       653   

                        cause_name  year  mortality_rate  
0               Transport injuries  2010       27.601971  
1          Substance use disorders  2014        1.407217  
2  Maternal and neonatal disorders  2014       78.941246  
3        Other infectious diseases  2014       48.133560  
4   Skin and subcutaneous diseases  2014        0.511941  


In [37]:
del df_ihme


In [38]:
df_country_reference = df_final[['country_name', 'country_code']].drop_duplicates().sort_values('country_name').reset_index(drop=True)
df_country_reference.to_csv("../Data/country_reference_list.csv", index=False)
print(df_country_reference.head())


     country_name country_code
0     Afghanistan          AFG
1         Albania          ALB
2         Algeria          DZA
3  American Samoa          ASM
4         Andorra          AND


In [39]:
# Step 1: Import the new dataset
df_gdp_health = pd.read_csv("../Data/Percentage of GDP spemt on Public Health.csv")

# Step 2: Rename columns for consistency
df_gdp_health.rename(columns={
    'Entity': 'country_name',
    'Code': 'country_code',
    'Year': 'year'
}, inplace=True)

# Step 3: Load your country reference list
df_country_reference = pd.read_csv("../Data/country_reference_list.csv")

# Step 4: Find unmatched country names
unique_new = set(df_gdp_health['country_name'].unique())
unique_ref = set(df_country_reference['country_name'].unique())
unmatched_countries = sorted(list(unique_new - unique_ref))

# Step 5: Show unmatched countries
for country in unmatched_countries:
    print( country)


South Korea
Turkey
United States


In [40]:
# Step 1: Define accurate country name and ISO3 code corrections
corrections = {
    "South Korea": ("Republic of Korea", "KOR"),
    "Turkey": ("Türkiye", "TUR"),
    "United States": ("United States of America", "USA")
}

# Step 2: Apply both name and code corrections
for original, (new_name, new_code) in corrections.items():
    df_gdp_health.loc[df_gdp_health['country_name'] == original, 'country_name'] = new_name
    df_gdp_health.loc[df_gdp_health['country_name'] == new_name, 'country_code'] = new_code

# Step 3: Preview the updated rows
print(df_gdp_health[df_gdp_health['country_name'].isin([val[0] for val in corrections.values()])][['country_name', 'country_code']].drop_duplicates())


                  country_name country_code
1805         Republic of Korea          KOR
2033                   Türkiye          TUR
2184  United States of America          USA


In [41]:
df_gdp_health.head()

,country_name,country_code,year,Public health expenditure as a share of GDP
0,Argentina,ARG,1880,0.0
1,Argentina,ARG,1890,0.0
2,Argentina,ARG,1900,0.0
3,Argentina,ARG,1910,0.0
4,Argentina,ARG,1920,0.0


In [42]:
# Step 1: Select only necessary columns
df_gdp_health_cleaned = df_gdp_health[[
    'country_code', 'year', 'Public health expenditure as a share of GDP'
]].rename(columns={
    'Public health expenditure as a share of GDP': 'percent_gdp_spent_on_healthcare'
})

# Step 2: Merge into a new preview DataFrame (not altering df_final yet)
df_preview = df_final.merge(df_gdp_health_cleaned, on=['country_code', 'year'], how='left')

# Step 3: Preview the merged DataFrame
print(df_preview.head())

   country_id country_name country_code  cause_id  \
0         160  Afghanistan          AFG       688   
1         160  Afghanistan          AFG       973   
2         160  Afghanistan          AFG       962   
3         160  Afghanistan          AFG       961   
4         160  Afghanistan          AFG       653   

                        cause_name  year  mortality_rate  \
0               Transport injuries  2010       27.601971   
1          Substance use disorders  2014        1.407217   
2  Maternal and neonatal disorders  2014       78.941246   
3        Other infectious diseases  2014       48.133560   
4   Skin and subcutaneous diseases  2014        0.511941   

   percent_gdp_spent_on_healthcare  
0                              NaN  
1                              NaN  
2                              NaN  
3                              NaN  
4                              NaN  


In [43]:
df_preview.to_csv("../Data/final.csv", index=False)

In [44]:
df_final=df_preview

In [45]:
# Step 1: Import the corruption dataset
df_corruption = pd.read_csv("../Data/political-corruption-index.csv")

# Step 2: Rename columns to stay consistent
df_corruption.rename(columns={
    'Entity': 'country_name',
    'Code': 'country_code',
    'Year': 'year',
    'Political corruption index (central estimate, aggregate: average)': 'corruption_index'
}, inplace=True)

# Step 4: Compare country names
unique_new = set(df_corruption['country_name'].unique())
unique_ref = set(df_country_reference['country_name'].unique())
unmatched_countries = sorted(list(unique_new - unique_ref))

# Step 5: Show unmatched country names
for country in unmatched_countries:
    print(country)


Africa
Asia
Baden
Bavaria
Bolivia
Brunswick
Cape Verde
Cote d'Ivoire
Democratic Republic of Congo
Democratic Republic of Vietnam
Duchy of Nassau
East Germany
East Timor
Europe
Hanover
Hesse Grand Ducal
Hong Kong
Iran
Kosovo
Laos
Mecklenburg Schwerin
Modena
Moldova
North America
North Korea
Oceania
Oldenburg
Palestine/Gaza
Palestine/West Bank
Parma
Piedmont-Sardinia
Republic of Vietnam
Russia
Saxe-Weimar-Eisenach
Saxony
Somaliland
South America
South Korea
Syria
Tanzania
Turkey
Tuscany
Two Sicilies
United States
Vatican
Venezuela
Vietnam
West Germany
World
Wurttemberg
Yemen Arab Republic
Yemen People's Republic
Zanzibar


In [46]:
# Step 3: Apply manual country name and code corrections
corruption_fixes = {
    "Africa": ("Central African Republic", "CAF"),
    "Bolivia": ("Bolivia (Plurinational State of)", "BOL"),
    "Cote d'Ivoire": ("Côte d'Ivoire", "CIV"),
    "Democratic Republic of Congo": ("Democratic Republic of the Congo", "COD"),
    "Democratic Republic of Vietnam": ("Viet Nam", "VNM"),
    "East Timor": ("Timor-Leste", "TLS"),
    "Iran": ("Iran (Islamic Republic of)", "IRN"),
    "Laos": ("Lao People's Democratic Republic", "LAO"),
    "Moldova": ("Republic of Moldova", "MDA"),
    "North Korea": ("Democratic People's Republic of Korea", "PRK"),
    "Palestine/Gaza": ("Palestine", "PSE"),
    "Palestine/West Bank": ("Palestine", "PSE"),
    "Republic of Vietnam": ("Viet Nam", "VNM"),
    "Russia": ("Russian Federation", "RUS"),
    "South Korea": ("Republic of Korea", "KOR"),
    "Syria": ("Syrian Arab Republic", "SYR"),
    "Tanzania": ("United Republic of Tanzania", "TZA"),
    "Turkey": ("Türkiye", "TUR"),
    "United States": ("United States of America", "USA"),
    "Vietnam": ("Viet Nam", "VNM"),
    "Yemen Arab Republic": ("Yemen", "YEM"),
    "Yemen People's Republic": ("Yemen", "YEM")
}

for original, (new_name, new_code) in corruption_fixes.items():
    df_corruption.loc[df_corruption['country_name'] == original, 'country_name'] = new_name
    df_corruption.loc[df_corruption['country_name'] == new_name, 'country_code'] = new_code

# Step 4: Filter down to only useful columns
df_corruption_cleaned = df_corruption[['country_code', 'year', 'corruption_index']]

# Step 5: Preview the cleaned dataframe
print(df_corruption_cleaned.head(10))

  country_code  year  corruption_index
0          AFG  1789             0.428
1          AFG  1790             0.428
2          AFG  1791             0.428
3          AFG  1792             0.428
4          AFG  1793             0.428
5          AFG  1794             0.428
6          AFG  1795             0.428
7          AFG  1796             0.428
8          AFG  1797             0.428
9          AFG  1798             0.428


In [47]:
# Merge corruption index into final dataset without saving
df_preview = df_final.merge(df_corruption_cleaned, on=['country_code', 'year'], how='left')

# Preview the result
print(df_preview.head())


   country_id country_name country_code  cause_id  \
0         160  Afghanistan          AFG       688   
1         160  Afghanistan          AFG       973   
2         160  Afghanistan          AFG       962   
3         160  Afghanistan          AFG       961   
4         160  Afghanistan          AFG       653   

                        cause_name  year  mortality_rate  \
0               Transport injuries  2010       27.601971   
1          Substance use disorders  2014        1.407217   
2  Maternal and neonatal disorders  2014       78.941246   
3        Other infectious diseases  2014       48.133560   
4   Skin and subcutaneous diseases  2014        0.511941   

   percent_gdp_spent_on_healthcare  corruption_index  
0                              NaN             0.958  
1                              NaN             0.927  
2                              NaN             0.927  
3                              NaN             0.927  
4                              NaN           

In [48]:
# Save df_preview to CSV
df_preview.to_csv("../Data/final.csv", index=False)

# Update df_final with the contents of df_preview
df_final = df_preview.copy()




In [49]:
import pandas as pd

# Step 1: Load reference country list
df_country_reference = pd.read_csv("../Data/country_reference_list.csv")

# Step 2: Load both datasets
df_physicians = pd.read_csv("../Data/physicians-per-1000-people.csv")
df_beds = pd.read_csv("../Data/hospital-beds-per-1000-people.csv")

# Step 3: Rename columns for consistency
df_physicians.rename(columns={
    'Entity': 'country_name',
    'Code': 'country_code',
    'Year': 'year',
    'Physicians (per 1,000 people)': 'physicians_per_1000'
}, inplace=True)

df_beds.rename(columns={
    'Entity': 'country_name',
    'Code': 'country_code',
    'Year': 'year',
    'Hospital beds (per 1,000 people)': 'beds_per_1000'
}, inplace=True)

# Step 4: Get reference set of countries
ref_countries = set(df_country_reference['country_name'].unique())

# Step 5: Unmatched countries in physicians dataset
physician_countries = set(df_physicians['country_name'].unique())
unmatched_physicians = sorted(physician_countries - ref_countries)

print("Unmatched Countries in Physicians Dataset:")
for country in unmatched_physicians:
    print(country)

# Step 6: Unmatched countries in beds dataset
beds_countries = set(df_beds['country_name'].unique())
unmatched_beds = sorted(beds_countries - ref_countries)

print("\nUnmatched Countries in Hospital Beds Dataset:")
for country in unmatched_beds:
    print(country)


Unmatched Countries in Physicians Dataset:
Aruba
Bolivia
Brunei
Cape Verde
Cayman Islands
Channel Islands
Cote d'Ivoire
Democratic Republic of Congo
East Asia and Pacific (WB)
East Timor
Europe and Central Asia (WB)
European Union (27)
French Polynesia
High-income countries
Hong Kong
Iran
Isle of Man
Laos
Latin America and Caribbean (WB)
Low-income countries
Lower-middle-income countries
Macao
Micronesia (country)
Middle East and North Africa (WB)
Middle-income countries
Moldova
New Caledonia
North America (WB)
North Korea
Russia
South Asia (WB)
South Korea
Sub-Saharan Africa (WB)
Syria
Tanzania
Turkey
United States
Upper-middle-income countries
Venezuela
Vietnam
World

Unmatched Countries in Hospital Beds Dataset:
Bolivia
Brunei
Cape Verde
Cayman Islands
Channel Islands
Cote d'Ivoire
Democratic Republic of Congo
East Asia and Pacific (WB)
East Timor
Europe and Central Asia (WB)
European Union (27)
High-income countries
Hong Kong
Iran
Isle of Man
Laos
Latin America and Caribbean (WB)
L

In [50]:
# Step 3: Define correction map
country_name_corrections = {
    "Brunei": ("Brunei Darussalam", "BRN"),
    "Micronesia (country)": ("Micronesia (Federated States of)", "FSM"),
    "Africa": ("Central African Republic", "CAF"),
    "Bolivia": ("Bolivia (Plurinational State of)", "BOL"),
    "Cote d'Ivoire": ("Côte d'Ivoire", "CIV"),
    "Democratic Republic of Congo": ("Democratic Republic of the Congo", "COD"),
    "Democratic Republic of Vietnam": ("Viet Nam", "VNM"),
    "East Timor": ("Timor-Leste", "TLS"),
    "Iran": ("Iran (Islamic Republic of)", "IRN"),
    "Laos": ("Lao People's Democratic Republic", "LAO"),
    "Moldova": ("Republic of Moldova", "MDA"),
    "North Korea": ("Democratic People's Republic of Korea", "PRK"),
    "Palestine/Gaza": ("Palestine", "PSE"),
    "Palestine/West Bank": ("Palestine", "PSE"),
    "Republic of Vietnam": ("Viet Nam", "VNM"),
    "Russia": ("Russian Federation", "RUS"),
    "South Korea": ("Republic of Korea", "KOR"),
    "Syria": ("Syrian Arab Republic", "SYR"),
    "Tanzania": ("United Republic of Tanzania", "TZA"),
    "Turkey": ("Türkiye", "TUR"),
    "United States": ("United States of America", "USA"),
    "Vietnam": ("Viet Nam", "VNM"),
    "Yemen Arab Republic": ("Yemen", "YEM"),
    "Yemen People's Republic": ("Yemen", "YEM")
}

# Step 4: Apply corrections to both dataframes
def apply_country_corrections(df, name_col="country_name", code_col="country_code"):
    for old_name, (new_name, new_code) in country_name_corrections.items():
        df.loc[df[name_col] == old_name, name_col] = new_name
        df.loc[df[name_col] == new_name, code_col] = new_code
    return df

df_physicians = apply_country_corrections(df_physicians)
df_beds = apply_country_corrections(df_beds)

# Step 5: Merge on consistent keys
df_healthcare_capacity = pd.merge(
    df_physicians,
    df_beds,
    on=["country_name", "country_code", "year"],
    how="outer"
)

# Optional: Preview the merged result
print(df_healthcare_capacity.head())


  country_name country_code  year  physicians_per_1000  beds_per_1000
0  Afghanistan          AFG  1960                0.035       0.170627
1  Afghanistan          AFG  1965                0.063            NaN
2  Afghanistan          AFG  1970                0.065       0.199000
3  Afghanistan          AFG  1981                0.077       0.275600
4  Afghanistan          AFG  1986                0.183            NaN


In [51]:
df_preview.head()

,country_id,country_name,country_code,cause_id,cause_name,year,mortality_rate,percent_gdp_spent_on_healthcare,corruption_index
0,160,Afghanistan,AFG,688,Transport injuries,2010,27.601971,NaN,0.958
1,160,Afghanistan,AFG,973,Substance use disorders,2014,1.407217,NaN,0.927
2,160,Afghanistan,AFG,962,Maternal and neonatal disorders,2014,78.941246,NaN,0.927
3,160,Afghanistan,AFG,961,Other infectious diseases,2014,48.133560,NaN,0.927
4,160,Afghanistan,AFG,653,Skin and subcutaneous diseases,2014,0.511941,NaN,0.927


In [52]:
# Merge df_healthcare_capacity into df_final
df_preview = df_final.merge(
    df_healthcare_capacity[["country_code", "year", "physicians_per_1000", "beds_per_1000"]],
    on=["country_code", "year"],
    how="left"
)

# Preview the result
print(df_preview.head())



   country_id country_name country_code  cause_id  \
0         160  Afghanistan          AFG       688   
1         160  Afghanistan          AFG       973   
2         160  Afghanistan          AFG       962   
3         160  Afghanistan          AFG       961   
4         160  Afghanistan          AFG       653   

                        cause_name  year  mortality_rate  \
0               Transport injuries  2010       27.601971   
1          Substance use disorders  2014        1.407217   
2  Maternal and neonatal disorders  2014       78.941246   
3        Other infectious diseases  2014       48.133560   
4   Skin and subcutaneous diseases  2014        0.511941   

   percent_gdp_spent_on_healthcare  corruption_index  physicians_per_1000  \
0                              NaN             0.958                0.245   
1                              NaN             0.927                0.304   
2                              NaN             0.927                0.304   
3           

In [53]:
# Save df_preview to CSV
df_preview.to_csv("../Data/final.csv", index=False)

# Update df_final with the contents of df_preview
df_final = df_preview.copy()




In [54]:
# Load reference country list
df_country_reference = pd.read_csv("../Data/country_reference_list.csv")

# Load happiness dataset
df_happiness = pd.read_csv("../Data/gdp-vs-happiness.csv")
df_happiness.rename(columns={
    'Entity': 'country_name',
    'Code': 'country_code',
    'Year': 'year',
    'Cantril ladder score': 'happinessscore'
}, inplace=True)

# Load healthcare expenditure dataset
df_expenditure = pd.read_csv("../Data/annual-healthcare-expenditure-per-capita.csv")
df_expenditure.rename(columns={
    'Entity': 'country_name',
    'Code': 'country_code',
    'Year': 'year',
    'Current health expenditure per capita, PPP (current international $)': 'health_expenditure_per_capita'
}, inplace=True)

# Get reference country set
ref_countries = set(df_country_reference['country_name'].unique())

# Compare happiness dataset
happiness_countries = set(df_happiness['country_name'].unique())
unmatched_happiness = sorted(happiness_countries - ref_countries)
print("Unmatched Countries in Happiness Dataset:")
for country in unmatched_happiness:
    print(country)

# Compare healthcare expenditure dataset
expenditure_countries = set(df_expenditure['country_name'].unique())
unmatched_expenditure = sorted(expenditure_countries - ref_countries)
print("\nUnmatched Countries in Healthcare Expenditure Dataset:")
for country in unmatched_expenditure:
    print(country)


Unmatched Countries in Happiness Dataset:
Africa
Aland Islands
Anguilla
Aruba
Asia
Austria-Hungary
Bolivia
Bonaire Sint Eustatius and Saba
British Indian Ocean Territory
British Virgin Islands
Brunei
Cape Verde
Cayman Islands
Christmas Island
Cocos Islands
Cote d'Ivoire
Curacao
Czechoslovakia
Democratic Republic of Congo
Democratic Republic of Vietnam
Duchy of Modena and Reggio
Duchy of Parma and Piacenza
East Asia and Pacific (WB)
East Germany
East Timor
Ethiopia (former)
Europe
Europe and Central Asia (WB)
European Union (27)
Faeroe Islands
Falkland Islands
Faroe Islands
Federal Republic of Central America
French Guiana
French Polynesia
Gibraltar
Grand Duchy of Baden
Grand Duchy of Tuscany
Great Colombia
Guadeloupe
Guernsey
High-income countries
Hong Kong
Iran
Isle of Man
Jersey
Kingdom of Bavaria
Kingdom of Sardinia
Kingdom of Saxony
Kingdom of Wurttemberg
Kingdom of the Two Sicilies
Korea (former)
Kosovo
Laos
Latin America and Caribbean (WB)
Liechtenstein
Low-income countries
Lower

In [55]:
country_name_corrections = {
    "Brunei": ("Brunei Darussalam", "BRN"),
    "Micronesia (country)": ("Micronesia (Federated States of)", "FSM"),
    "Africa": ("Central African Republic", "CAF"),
    "Bolivia": ("Bolivia (Plurinational State of)", "BOL"),
    "Cote d'Ivoire": ("Côte d'Ivoire", "CIV"),
    "Democratic Republic of Congo": ("Democratic Republic of the Congo", "COD"),
    "Democratic Republic of Vietnam": ("Viet Nam", "VNM"),
    "East Timor": ("Timor-Leste", "TLS"),
    "Iran": ("Iran (Islamic Republic of)", "IRN"),
    "Laos": ("Lao People's Democratic Republic", "LAO"),
    "Moldova": ("Republic of Moldova", "MDA"),
    "North Korea": ("Democratic People's Republic of Korea", "PRK"),
    "Palestine/Gaza": ("Palestine", "PSE"),
    "Palestine/West Bank": ("Palestine", "PSE"),
    "Republic of Vietnam": ("Viet Nam", "VNM"),
    "Russia": ("Russian Federation", "RUS"),
    "South Korea": ("Republic of Korea", "KOR"),
    "Syria": ("Syrian Arab Republic", "SYR"),
    "Tanzania": ("United Republic of Tanzania", "TZA"),
    "Turkey": ("Türkiye", "TUR"),
    "United States": ("United States of America", "USA"),
    "Vietnam": ("Viet Nam", "VNM"),
    "Yemen Arab Republic": ("Yemen", "YEM"),
    "Yemen People's Republic": ("Yemen", "YEM"),
    "Ethiopia (former)": ("Ethiopia", "ETH"),
    "Great Colombia": ("Colombia", "COL"),
    "Netherlands Antilles": ("Netherlands", "NLD"),
    "Northern Cyprus": ("Cyprus", "CYP"),
    "Pakistan (former)": ("Pakistan", "PAK"),
    "USSR": ("Russian Federation", "RUS"),
    "Venezuela": ("Venezuela (Bolivarian Republic of)", "VEN")
}

# Step 5: Apply corrections
def apply_country_corrections(df, name_col="country_name", code_col="country_code"):
    for old_name, (new_name, new_code) in country_name_corrections.items():
        df.loc[df[name_col] == old_name, name_col] = new_name
        df.loc[df[name_col] == new_name, code_col] = new_code
    return df

df_happiness = apply_country_corrections(df_happiness)
df_expenditure = apply_country_corrections(df_expenditure)

# Step 6: Merge
df_health_metrics = pd.merge(
    df_happiness,
    df_expenditure,
    on=["country_name", "country_code", "year"],
    how="outer"
)

# Drop unwanted columns if they exist
df_health_metrics.drop(columns=[
    "GDP per capita, PPP (constant 2021 international $)",
    "World regions according to OWID"
], inplace=True, errors='ignore')

# Preview the cleaned dataframe
print(df_health_metrics.head())


  country_name country_code  year  happinessscore  \
0  Afghanistan          AFG  2000             NaN   
1  Afghanistan          AFG  2001             NaN   
2  Afghanistan          AFG  2002             NaN   
3  Afghanistan          AFG  2003             NaN   
4  Afghanistan          AFG  2004             NaN   

   health_expenditure_per_capita  
0                            NaN  
1                            NaN  
2                      87.396550  
3                      86.260340  
4                      93.952965  


In [56]:
df_preview.head()

,country_id,country_name,country_code,cause_id,cause_name,year,mortality_rate,percent_gdp_spent_on_healthcare,corruption_index,physicians_per_1000,beds_per_1000
0,160,Afghanistan,AFG,688,Transport injuries,2010,27.601971,NaN,0.958,0.245,0.43
1,160,Afghanistan,AFG,973,Substance use disorders,2014,1.407217,NaN,0.927,0.304,0.46
2,160,Afghanistan,AFG,962,Maternal and neonatal disorders,2014,78.941246,NaN,0.927,0.304,0.46
3,160,Afghanistan,AFG,961,Other infectious diseases,2014,48.133560,NaN,0.927,0.304,0.46
4,160,Afghanistan,AFG,653,Skin and subcutaneous diseases,2014,0.511941,NaN,0.927,0.304,0.46


In [57]:
# Merge df_health_metrics into df_preview
df_preview = df_preview.merge(
    df_health_metrics[["country_code", "year", "happinessscore", "health_expenditure_per_capita"]],
    on=["country_code", "year"],
    how="left"
)

# Preview the result
print(df_preview.head())

   country_id country_name country_code  cause_id  \
0         160  Afghanistan          AFG       688   
1         160  Afghanistan          AFG       973   
2         160  Afghanistan          AFG       962   
3         160  Afghanistan          AFG       961   
4         160  Afghanistan          AFG       653   

                        cause_name  year  mortality_rate  \
0               Transport injuries  2010       27.601971   
1          Substance use disorders  2014        1.407217   
2  Maternal and neonatal disorders  2014       78.941246   
3        Other infectious diseases  2014       48.133560   
4   Skin and subcutaneous diseases  2014        0.511941   

   percent_gdp_spent_on_healthcare  corruption_index  physicians_per_1000  \
0                              NaN             0.958                0.245   
1                              NaN             0.927                0.304   
2                              NaN             0.927                0.304   
3           

In [58]:
df_preview.to_csv("../Data/final.csv", index=False)
df_final = df_preview.copy()

In [2]:
df_final = pd.read_csv("final.csv")

# Using Cautious fill for happiness score column
def cautious_fill(series, threshold=1.0):
    ffill = series.ffill()
    bfill = series.bfill()
    result = series.copy()
    for i in range(len(series)):
        if pd.isnull(series.iloc[i]):
            prev_val = ffill.iloc[i]
            next_val = bfill.iloc[i]
            if pd.notnull(prev_val) and pd.notnull(next_val):
                if abs(prev_val - next_val) <= threshold:
                    result.iloc[i] = (prev_val + next_val) / 2
            elif pd.notnull(prev_val):
                result.iloc[i] = prev_val
            elif pd.notnull(next_val):
                result.iloc[i] = next_val
    return result

# To fill missing Value 

def fill_missing_values(df, cautious_column='happinessscore', cautious_threshold=2.0,
                        standard_columns=None):
    if standard_columns is None:
        standard_columns = [
            "beds_per_1000",
            "physicians_per_1000",
            "percent_gdp_spent_on_healthcare",
            "health_expenditure_per_capita"
        ]

    
    df = df.sort_values(by=["country_name", "year"]).copy()

    # Apply cautious fill to selected column
    df[cautious_column] = df.groupby("country_name")[cautious_column].transform(
        lambda x: cautious_fill(x, threshold=cautious_threshold)
    )

    # Standard ffill + bfill for other columns
    for col in standard_columns:
        df[col] = df.groupby("country_name")[col].transform(lambda x: x.ffill().bfill())

    return df

# Apply the filling process
df_final = fill_missing_values(df_final)

In [1]:
import pandas as pd